# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [16]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'KOZVZUGTaASpXbqEDs3AY195hM61GbyJYoPX7TBP'  # Replaced by my own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags', 'duration']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [17]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 20]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [18]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'bell',
        'filter': 'duration:[1 TO 3]',
        'num_results': 20, 
    },
    {
        'query': 'drum',
        'filter': 'duration:[1 TO 3]',
        'num_results': 20, 
    },
    {
        'query': 'speech',
        'filter': 'duration:[1 TO 3]',
        'num_results': 20, 
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 60 entries! dataframe.csv


,duration,freesound_id,license,name,path,tags,username
0,2.316190,333693,http://creativecommons.org/publicdomain/zero/1.0/,Thin bell ding 4,files/333693_1187042-hq.ogg,"[ding, chime, short, dong, bell]",Khrinx
1,2.536463,30160,http://creativecommons.org/licenses/by/3.0/,BellTinyShake.wav,files/30160_129090-hq.ogg,"[bell, small]",HerbertBoland
2,1.999600,216305,http://creativecommons.org/licenses/by/3.0/,Calling_Bell_03.wav,files/216305_3761494-hq.ogg,"[Antique, Plate, Ringing, Bell]",RSilveira_88
3,1.334650,173000,http://creativecommons.org/licenses/by/3.0/,Bicycle bell 2.wav,files/173000_2193194-hq.ogg,"[chime, bicycle, bell]",KeyKrusher
4,1.477170,318193,http://creativecommons.org/licenses/by-nc/3.0/,05969 common bicycle bell.wav,files/318193_321967-hq.ogg,"[bicycle, bell, warn, bike, common, ring]",Robinhood76
5,2.917150,383593,http://creativecommons.org/publicdomain/zero/1.0/,bicycle-bell_04.wav,files/383593_4067257-hq.ogg,"[contact, metal, glockenspiel, clang, high-pit...",newagesoup
6,2.213250,434818,http://creativecommons.org/licenses/by/3.0/,Bike bell,files/434818_1992856-hq.ogg,"[bells, ring, bike, bell, ringing, bicycle, go...",dersuperanton
7,2.673990,323129,http://creativecommons.org/publicdomain/zero/1.0/,Bicycle_Bell_from_BikeKitchen_Augsburg_09.wav,files/323129_4438201-hq.ogg,"[bicycle, bell, street, metallic, bike, mechan...",geraldfiebig
8,1.109190,258179,http://creativecommons.org/licenses/by/3.0/,BellPing2.flac,files/258179_4486188-hq.ogg,"[ding, ringing, bell, 706, ping, telephone, ri...",kwahmah_02
9,1.088210,456565,http://creativecommons.org/licenses/by/3.0/,bell1.wav,files/456565_5052309-hq.ogg,"[bell, electronic, chime]",bumpelsnake
